# Exploring and clustering Neighbourhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Read Toronto dataset with Latitude and Longitude

In [2]:
Toronto_post = pd.read_csv("TorontoPostData.csv")
Toronto_post.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Get only the Neighbourhoods from Scarborough for exploring

In [3]:
Scarborough_data = Toronto_post[Toronto_post['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Get geographical coordinates of Toronto

In [4]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


Visualize Scarborough and the neighbourhoods in it

In [5]:
# create map of Scarborough using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

Get foursquare credentials and version

In [6]:
CLIENT_ID = 'F2R520QHLU0J1BI04AVCKQNKV1EAW1NX1MJQTKGMZE0CIQTZ' # your Foursquare ID
CLIENT_SECRET = 'U4ZLNVLE42DHCP2RIUTMV2XZBRV4L4KCA2AU3QYG0DU1TTP2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F2R520QHLU0J1BI04AVCKQNKV1EAW1NX1MJQTKGMZE0CIQTZ
CLIENT_SECRET:U4ZLNVLE42DHCP2RIUTMV2XZBRV4L4KCA2AU3QYG0DU1TTP2


Explore the fifth neighbourhood in Scarborough

In [7]:
Scarborough_data.loc[4, 'Neighbourhood']

'Cedarbrae '

Get the latitude and longitude values of the neigbourhood

In [8]:
neighborhood_latitude = Scarborough_data.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[4, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cedarbrae  are 43.773136, -79.23947609999998.


Get the top 100 venues within 500 miles radius of Cedarbrae

In [9]:
#set the limit to be returned by Foursquare API
LIMIT = 100 

#define the radius
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
#--> 

'https://api.foursquare.com/v2/venues/explore?&client_id=F2R520QHLU0J1BI04AVCKQNKV1EAW1NX1MJQTKGMZE0CIQTZ&client_secret=U4ZLNVLE42DHCP2RIUTMV2XZBRV4L4KCA2AU3QYG0DU1TTP2&v=20180605&ll=43.773136,-79.23947609999998&radius=500&limit=100'

In [10]:
results = requests.get(url).json()
#results

Get category from the json

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure in dataframe

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
2,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
3,Thai One On,Thai Restaurant,43.774468,-79.241268
4,TD Canada Trust,Bank,43.774952,-79.241343


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


### Explore Neighbourhoods in Scarborough

Create a function to repeat the process for all the neighbourhoods in Scarborough

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Code to run the function for each neighbourhood in Scarborough and put in dataframe

In [15]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighbourhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge , Malvern 
Highland Creek , Rouge Hill , Port Union 
Guildwood , Morningside , West Hill 
Woburn 
Cedarbrae 
Scarborough Village 
East Birchmount Park , Ionview , Kennedy Park 
Clairlea , Golden Mile , Oakridge 
Cliffcrest , Cliffside , Scarborough Village West 
Birch Cliff , Cliffside West 
Dorset Park , Scarborough Town Centre , Wexford Heights 
Maryvale , Wexford 
Agincourt 
Clarks Corners , Sullivan , Tam O'Shanter 
Agincourt North , L'Amoreaux East , Milliken , Steeles East 
L'Amoreaux West 
Upper Rouge 


Check the size of the resulting dataframe

In [16]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(89, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge , Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge , Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


Check how many venues for each neighbourhood

In [17]:
Scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North , L'Amoreaux East , Milliken , Steeles East",3,3,3,3,3,3
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea , Golden Mile , Oakridge",10,10,10,10,10,10
"Clarks Corners , Sullivan , Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest , Cliffside , Scarborough Village West",3,3,3,3,3,3
"Dorset Park , Scarborough Town Centre , Wexford Heights",6,6,6,6,6,6
"East Birchmount Park , Ionview , Kennedy Park",4,4,4,4,4,4


Find out how many unique categories from the returned venues

In [18]:
print('There are {} unique categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 53 unique categories.


### Analyze each neighbourhood

In [19]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
Scarborough_onehot['Neighbourhood'] = Scarborough_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge , Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge , Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek , Rouge Hill , Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek , Rouge Hill , Port Union",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


### Group rows by neigbourhood by taking the mean of the frequency of occurence in each category

In [20]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
Scarborough_grouped

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Tech Startup,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.250000,0.0,0.0,0.00,0.000,0.250000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.250,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
1,"Agincourt North , L'Amoreaux East , Milliken ,...",0.000000,0.333333,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
2,"Birch Cliff , Cliffside West",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.25,0.000,0.000000,0.000000,0.25,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.25,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000,0.000000,0.000000
3,Cedarbrae,0.000000,0.000000,0.125,0.000000,0.125000,0.125,0.0,0.000000,0.0,0.0,0.00,0.125,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.125,0.00,0.000000,0.125,0.000000,0.000000,0.0,0.00,0.000000,0.125,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.000000
4,"Clairlea , Golden Mile , Oakridge",0.000000,0.000000,0.000,0.000000,0.200000,0.000,0.0,0.000000,0.2,0.1,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.100000,0.000,0.00,0.000000,0.000,0.000000,0.100000,0.0,0.00,0.000000,0.000,0.000000,0.1,0.000000,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.000000,0.000,0.000000,0.000000
5,"Clarks Corners , Sullivan , Tam O'Shanter",0.000000,0.000000,0.000,0.000000,0.000000,0.100,0.0,0.000000,0.0,0.0,0.00,0.000,0.100000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.100000,0.100,0.00,0.000000,0.000,0.000000,0.000000,0.1,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.100000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.100,0.000000,0.000000
6,"Cliffcrest , Cliffside , Scarborough Village W...",0.333333,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.000000
7,"Dorset Park , Scarborough Town Centre , Wexfor...",0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.00,0.000,0.166667,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.333333,0.000000,0.0,0.00,0.166667,0.000,0.000000,0.0,0.000000,0.000000,0.0

Print the top 5 venues for each neighbourhood in Scarborough

In [21]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt ----
                 venue  freq
0               Lounge  0.25
1       Breakfast Spot  0.25
2       Sandwich Place  0.25
3   Chinese Restaurant  0.25
4  American Restaurant  0.00


----Agincourt North , L'Amoreaux East , Milliken , Steeles East ----
               venue  freq
0   Asian Restaurant  0.33
1               Park  0.33
2         Playground  0.33
3           Pharmacy  0.00
4  Korean Restaurant  0.00


----Birch Cliff , Cliffside West ----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4            Pizza Place  0.00


----Cedarbrae ----
                  venue  freq
0  Caribbean Restaurant  0.12
1    Athletics & Sports  0.12
2                Bakery  0.12
3                  Bank  0.12
4       Thai Restaurant  0.12


----Clairlea , Golden Mile , Oakridge ----
           venue  freq
0         Bakery   0.2
1       Bus Line   0.2
2   Intersection   0.1
3  Metro S

Write a function to sort the venues in descending order

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a data frame to display the top 10 most common venues for each neigbourhood

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Scarborough_grouped['Neighbourhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North , L'Amoreaux East , Milliken ,...",Park,Asian Restaurant,Playground,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff , Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
4,"Clairlea , Golden Mile , Oakridge",Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment


### Clustering of the Neighbourhoods

Run K-means to cluster neighbourhoods, use 5 clusters

In [24]:
# set number of clusters
kclusters = 5

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

Create a new dataframe which includes the cluster and the top 10 venues for each neigbourhood

In [25]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged = Scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how="inner")

Scarborough_merged.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,3,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighbourhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine each clusters

Cluster 1

In [27]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
14,Scarborough,0,Park,Asian Restaurant,Playground,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


The most common venues for cluster 1 are Playground and park

Cluster 2

In [28]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Intersection,Breakfast Spot,Mexican Restaurant,Tech Startup,Spa,Electronics Store,Medical Center,Pizza Place,Rental Car Location,College Stadium
3,Scarborough,1,Coffee Shop,Korean Restaurant,Convenience Store,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,Scarborough,1,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
6,Scarborough,1,Discount Store,Department Store,Playground,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,Scarborough,1,Bakery,Bus Line,Intersection,Fast Food Restaurant,Soccer Field,Bus Station,Metro Station,Park,College Stadium,General Entertainment
9,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Pet Store,Bakery,Grocery Store,General Entertainment,Fried Chicken Joint,Asian Restaurant
11,Scarborough,1,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint
12,Scarborough,1,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,1,Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Bank,Chinese Restaurant,Pharmacy,Coffee Shop


Majority of the neighbourhoods in cluster 2 seems to have restaurant as the most common venue

Cluster 3

In [29]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


For cluster 3, the most common venue is fast food restaurant

Cluster 4

In [30]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


For cluster 4, the most common venue is Bar

Cluster 5

In [31]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 4, Scarborough_merged.columns[[1] + list(range(5, Scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4,Motel,American Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


For cluster 5, the most common venue is Motel